In [1]:
# Import the basic spark library

from pyspark.sql import SparkSession

# Create an entry point to the PySpark Application
spark = SparkSession.builder \
      .master("local") \
      .appName("schemaBook") \
      .getOrCreate()
# master contains the URL of your remote spark instance or 'local'

22/12/07 02:16:54 WARN Utils: Your hostname, MacBook-Pro-di-Cristina.local resolves to a loopback address: 127.0.0.1; using 172.20.10.2 instead (on interface en0)
22/12/07 02:16:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/07 02:16:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/07 02:16:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/12/07 02:16:56 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/12/07 02:16:56 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/12/07 02:16:56 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/12/07 02:16:56 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
22/12/07 02:16:56 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
22/12/07 02:16:56 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
22/12/07 02:16:56 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.
22/12/07 02:16:56 WARN Utils: Service 'SparkUI' could not bind on

In [2]:
#schema book
df_book = spark.read.option("multiline","true").json("book-db-manual.json")
df_book.printSchema()
#df_book.show()

root
 |-- author: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- citations: long (nullable = true)
 |-- crossref: string (nullable = true)
 |-- editor: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- ee: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- isbn: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- key: string (nullable = true)
 |-- keyword: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- pages: long (nullable = true)
 |-- publisher: string (nullable = true)
 |-- series: struct (nullable = true)
 |    |-- href: string (nullable = true)
 |    |-- title: string (nullable = true)
 |-- title: string (nullable = true)
 |-- volume: long (nullable = true)
 |-- 

In [3]:
#schema article
df_article = spark.read.option("multiline","true").json("article-db-manual_conrefs.json")
df_article.printSchema()
#df_article.show()

root
 |-- author: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- citations: long (nullable = true)
 |-- cite: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ee: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- journal: string (nullable = true)
 |-- key: string (nullable = true)
 |-- keyword: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- note: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- pages: long (nullable = true)
 |-- publisher: string (nullable = true)
 |-- title: string (nullable = true)
 |-- volume: long (nullable = true)
 |-- year: long (nullable = true)



In [4]:
#schema www
df_www = spark.read.option("multiline","true").json("www-db-manual.json")
df_www.printSchema()
#df_www.show()

root
 |-- author: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- orcid: string (nullable = true)
 |-- key: string (nullable = true)
 |-- note: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- url: array (nullable = true)
 |    |-- element: string (containsNull = true)



0.1.4 VERSION 2: Best PROFESSORS according to number of citations at a given university

The following query can be used to find the authors affiliated with a specific university
(e.g.’Stanford University, USA’) ordering them by the average number of citations(at least 5) received by their articles from any other publication in the last years(from 2000).

In [5]:
#import
from pyspark.sql.functions import col                       # Filtering using the col() function
from pyspark.sql.functions import array_contains            # Filtering on array columns
from pyspark.sql.functions import explode                   # Explode Arrays in Individual Rows
from pyspark.sql.functions import collect_list
from pyspark.sql.functions import avg, count                # Multiple Aggregations



In [6]:
selected_df_book = df_book  \
    .select(df_book.key, df_book.year)
#exploded_df_book.printSchema()
#exploded_df_book.show(truncate=False)


In [7]:
selected_df_article = df_article  \
    .select(df_article.key, df_article.author, df_article.cite, df_article.year )
#selected_df_article.printSchema()
#selected_df_article.show(truncate=False)

In [8]:
filtered_df_www=df_www.filter(array_contains(df_www.note, "Stanford University, USA")).select(explode(df_www.author)).withColumnRenamed("col", "author")
filtered_df_www.printSchema()
filtered_df_www.show(truncate=False)

root
 |-- author: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- orcid: string (nullable = true)

+--------------------------------------------------+
|author                                            |
+--------------------------------------------------+
|{Jennifer Widom, Jennifer Widom}                  |
|{Gio Wiederhold, Gio Wiederhold}                  |
|{Janet L. Wiener, Janet L. Wiener}                |
|{Terry Winograd, Terry Winograd}                  |
|{William J. Dally, William J. Dally}              |
|{William (Bill) J. Dally, William (Bill) J. Dally}|
|{Bill Dally, Bill Dally}                          |
|{Jeffrey D. Ullman, Jeffrey D. Ullman}            |
|{Yue Zhuge, Yue Zhuge}                            |
|{Prabhakar Raghavan, Prabhakar Raghavan}          |
|{Alexander Aiken, Alexander Aiken}                |
|{Alex Aiken, Alex Aiken}                          |
|{Sergey Brin, Sergey Brin}                        |
|{Wilburt Labio, Wilburt 

In [9]:
df_pub= selected_df_article.unionByName(selected_df_book, allowMissingColumns=True).filter(col("year")>2000).withColumnRenamed("key", "key_p")
#df_pub.printSchema()
#df_pub.show(truncate=False)

In [10]:
exploded_df_article=selected_df_article.select(selected_df_article.key, selected_df_article.author, explode(selected_df_article.cite)).withColumnRenamed("col", "cite")
#exploded_df_article.printSchema()
#exploded_df_article.show(truncate=False)

In [11]:
exploded_df_article=exploded_df_article.join(df_pub,exploded_df_article.cite==df_pub.key_p, "left").select(exploded_df_article.key, explode(exploded_df_article.author), exploded_df_article.cite,df_pub.key_p).filter(df_pub.key_p.isNotNull()).drop(df_pub.key_p).withColumnRenamed("col", "author").groupBy("key").agg(count("cite").alias("CountOfCitations"),collect_list("author").alias("author")).select(col("key"), col("CountOfCitations"), explode(col("author"))).withColumnRenamed("col", "author")
#exploded_df_article.printSchema()
#exploded_df_article.show(truncate=False)

In [12]:
result = filtered_df_www.join(exploded_df_article, filtered_df_www.author == exploded_df_article.author, "left").drop(exploded_df_article.author).groupBy("author").agg(avg("CountOfCitations").alias("Avg Cit")).filter(col("Avg Cit")>="5").select(col("author.name"), col("Avg Cit")).sort(col("Avg Cit").desc())

result.printSchema()
result.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- Avg Cit: double (nullable = true)



+--------------------+------------------+
|name                |Avg Cit           |
+--------------------+------------------+
|Yue Zhuge           |40.0              |
|Wilburt Labio       |40.0              |
|Prabhakar Raghavan  |30.352941176470587|
|Janet L. Wiener     |24.0              |
|Andreas Paepcke     |21.0              |
|Jeffrey D. Ullman   |19.23076923076923 |
|Jennifer Widom      |18.01818181818182 |
|Hector Garcia-Molina|16.11111111111111 |
|Avi Pfeffer         |16.0              |
|William J. Dally    |14.0              |
|Gio Wiederhold      |13.393939393939394|
+--------------------+------------------+

